In [1]:
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn.linear_model import LinearRegression

from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation
import sklearn
from sklearn import pipeline, preprocessing, feature_extraction
import xgboost as xgb
from sklearn.metrics import f1_score
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import time
import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, auc, roc_curve
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from scipy import sparse
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgbm
import scipy
from sklearn.preprocessing import StandardScaler

%matplotlib inline
%pylab inline

/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


In [48]:
X_train = pd.read_csv("data/train.csv")
X_test = pd.read_csv("data/test_with_ids.csv")

y_train = pd.DataFrame(X_train['is_duplicate'])
y_train.columns = ['y']
del X_train['is_duplicate']

skf = StratifiedKFold(5, shuffle=True, random_state=0)
folds = []
for itr, ite in skf.split(X_train, y_train.y):
    folds += [[itr, ite]]

In [49]:
X_train = pd.read_csv("data/train_inter.csv")

X_train['question1'] = X_train['question1'].apply(lambda x: str(x))
X_train['question2'] = X_train['question2'].apply(lambda x: str(x))

X_train['question1'] = X_train['question1'].apply(lambda x: x.lower().split())
X_train['question2'] = X_train['question2'].apply(lambda x: x.lower().split())



In [50]:
X_test = pd.read_csv("data/test_inter.csv")

X_test['question1'] = X_test['question1'].apply(lambda x: str(x))
X_test['question2'] = X_test['question2'].apply(lambda x: str(x))

X_test['question1'] = X_test['question1'].apply(lambda x: x.lower().split())
X_test['question2'] = X_test['question2'].apply(lambda x: x.lower().split())

In [51]:
X = map(lambda x: ' '.join(x), X_train['question1'])
X1 = map(lambda x: ' '.join(x), X_train['question2'])
Y = map(lambda x: ' '.join(x), X_test['question1'])
Y1 = map(lambda x: ' '.join(x), X_test['question2'])
hw = HashingVectorizer(n_features=200).fit(X_train['question1'] + X_train['question2'])


In [52]:
X = pd.DataFrame(hw.transform(X).todense())
X1 = pd.DataFrame(hw.transform(X1).todense())
Y = pd.DataFrame(hw.transform(Y).todense())
Y1 = pd.DataFrame(hw.transform(Y1).todense())

In [53]:
tfidf = feature_extraction.text.TfidfTransformer()
tfidf.fit(X+X1)

X = pd.DataFrame(tfidf.transform(X).todense())
X1 = pd.DataFrame(tfidf.transform(X1).todense())
Y = pd.DataFrame(tfidf.transform(Y).todense())
Y1 = pd.DataFrame(tfidf.transform(Y1).todense())

In [54]:
X_train = pd.concat([X_train, X], axis=1, join_axes=[X_train.index])
del X
X_train = pd.concat([X_train, X1], axis=1, join_axes=[X_train.index])
del X1
X_test = pd.concat([X_test, Y], axis=1, join_axes=[X_test.index])
del Y
X_test = pd.concat([X_test, Y1], axis=1, join_axes=[X_test.index])
del Y1

In [24]:
del X_test


In [55]:
cores_dict = pd.read_csv("data/question_max_kcores.csv", index_col="qid").to_dict()["max_kcore"]
def gen_qid1_max_kcore(row):
    return cores_dict[row["qid1"]]
def gen_qid2_max_kcore(row):
    return cores_dict[row["qid2"]]

#def gen_max_kcore(row):
#    return max(row["qid1_max_kcore"], row["qid2_max_kcore"])

X_train["qid1_max_kcore"] = X_train.apply(gen_qid1_max_kcore, axis=1)
#X_test["qid1_max_kcore"] = X_test.apply(gen_qid1_max_kcore, axis=1)
X_train["qid2_max_kcore"] = X_train.apply(gen_qid2_max_kcore, axis=1)
#X_test["qid2_max_kcore"] = X_test.apply(gen_qid2_max_kcore, axis=1)

In [56]:
q_train = pd.read_csv("data/kcore_test.csv")
X_test = pd.concat([X_test, q_train], axis=1)

In [58]:
X_train[:2]

,id,qid1,qid2,question1,question2,word_match,tfidf_word_match,0,1,2,...,192,193,194,195,196,197,198,199,qid1_max_kcore,qid2_max_kcore
0,0,1,2,"[what, is, the, step, by, step, guide, to, inv...","[what, is, the, step, by, step, guide, to, inv...",0.727273,0.772164,0.0,0.0,0.0,...,0.325385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,1,3,4,"[what, is, the, story, of, kohinoor, (koh-i-no...","[what, would, happen, if, the, indian, governm...",0.307692,0.361758,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [59]:
del X_train['question1']
del X_train['question2']
del X_train['qid1']
del X_train['qid2']
del X_train['id']

del X_test['question1']
del X_test['question2']
del X_test['test_id']

In [68]:
del X_train

In [60]:
#X_train = pd.read_csv("data/train_inter.csv")
q_train = pd.read_csv("data/freq_train.csv")
X_train = pd.concat([X_train, q_train], axis=1)

f_train = pd.read_csv("data/train_features.csv", encoding = "ISO-8859-1")
f_train = f_train.ix[:, 2:]
X_train = pd.concat([X_train, f_train], axis=1)

In [61]:
q_train = pd.read_csv("data/f1_train.csv")
X_train = pd.concat([X_train, q_train], axis=1)

In [60]:
q_train = pd.read_csv("data/f2_train.csv")
X_train = pd.concat([X_train, q_train], axis=1)

In [62]:
q_train = pd.read_csv("data/train_lda_feat.csv", encoding = "ISO-8859-1")
q_train = q_train.ix[:, 6:]
X_train = pd.concat([X_train, q_train], axis=1)

In [70]:
X_test[:1]

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,192,193,194,195,196,197,198,199,qid1_max_kcore,qid2_max_kcore
0,0.266667,0.274019,0.0,0.0,0.0,0.0,0.0,-0.156312,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [67]:
X_train[:2]

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,kur_q1vec,kur_q2vec,q1_q2_intersect,cosine_distance_lda,cityblock_distance_lda,jaccard_distance_lda,canberra_distance_lda,euclidean_distance_lda,minkowski_distance_lda,braycurtis_distance_lda
0,0.727273,0.772164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.050416,0.337301,0,0.046653,0.232798,1.0,1.266667,0.137590,0.127564,0.134090
1,0.307692,0.361758,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.284010,-0.034444,0,0.748658,1.241746,1.0,5.400000,0.538923,0.423051,0.812442


In [96]:
import gc
gc.collect()

407

In [95]:
del q_test

In [71]:
q_test = pd.read_csv("data/freq_test.csv")


In [72]:
X_test = pd.concat([X_test, q_test], axis=1)

In [74]:
f_test = pd.read_csv("data/test_features.csv", encoding = "ISO-8859-1")
f_test = f_test.ix[:, 2:]

In [75]:
X_test = pd.concat([X_test, f_test], axis=1)

In [77]:
q_test = pd.read_csv("data/f1_test.csv")
X_test = pd.concat([X_test, q_test], axis=1)

In [19]:
q_test = pd.read_csv("data/f2_test.csv")
X_test = pd.concat([X_test, q_test], axis=1)

In [88]:
q_test = pd.read_csv("data/test_lda_feat.csv", encoding = "ISO-8859-1")
q_test = q_test.ix[:, 3:]
X_test = pd.concat([X_test, q_test], axis=1)

In [87]:
q_test.ix[:1, 3:]

,cosine_distance_lda,cityblock_distance_lda,jaccard_distance_lda,canberra_distance_lda,euclidean_distance_lda,minkowski_distance_lda,braycurtis_distance_lda
0,0.573341,1.160190,1.0,7.833784,0.399663,0.290631,0.65838
1,0.251020,0.615409,1.0,3.595264,0.265033,0.210156,0.35913


In [90]:
X_test[:1]

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,kur_q1vec,kur_q2vec,q1_q2_intersect,cosine_distance_lda,cityblock_distance_lda,jaccard_distance_lda,canberra_distance_lda,euclidean_distance_lda,minkowski_distance_lda,braycurtis_distance_lda
0,0.266667,0.274019,0.0,0.0,0.0,0.0,0.0,-0.156312,0.0,0.0,...,0.039938,-0.144866,0,0.573341,1.16019,1.0,7.833784,0.399663,0.290631,0.65838


In [125]:
param = {}
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'logloss'
param['max_depth'] = 7
# param['booster'] = 'dart'
#param['booster'] = 'gblinear'
param['eta'] = 0.1
#param['subsample'] = 1.0
# param["scale_pos_weight"] = 0.5
#param['colsample_bytree'] = 0.7
param['nthread'] = 8
#param['num_class'] =3
#param['alpha'] = 0.00
#param['lambda_bias'] = 0.1
# param['lambda'] = 0
#param['min_child_weight'] = 100

# if param['booster'] == 'gblinear': param['eta'] *= 1e-4
numround = 2000

#LGBM
params = {
    #'task': 'train',
    #'boosting_type': 'gbdt',
    #'objective': 'regression',
    #'metric': {'l2', 'auc'},
    #'num_leaves': 31,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    #'subsample': 0.8,
    #'colsample_bytree': 0.7,
    'objective': 'binary',
    'nthread': -1,
    'learning_rate': 0.1,
    'min_child_weight': 100,
    #'max_depth': 6
    #'num_class': 3
    
}

pred_train = np.zeros(len(y_train))
xgbs = []
sc,sc_mean = [],[]

for itr, ite in folds:
    ypred = []
    for i in range(1):
               
        param['seed'] = i+1
        
        #xgboost
        '''
        Xdatatrain = xgb.DMatrix(data=X_train.ix[itr, :].values,
                                     label=y_train.ix[itr].values)
        Xdataval = xgb.DMatrix(data=X_train.ix[ite, :].values,
                                     label=y_train.ix[ite].values)

        plst = list(param.items())
        watchlist = [(Xdatatrain, 'train'), (Xdataval, 'eval')]
        bst = xgb.train(plst, Xdatatrain, numround, evals=watchlist, verbose_eval=15)

        
        #rc = ensemble.ExtraTreesClassifier(n_estimators=1300, criterion='gini', max_depth=None, n_jobs=-1)
        #pred_train[ite] = bst.predict(Xdataval)
        #neigh = KNeighborsClassifier(n_neighbors=2)
        #neigh.fit(X_train.ix[itr, :], y_train.ix[itr])
        ypred = bst.predict(Xdataval)
        #ypred = neigh.predict_proba(X_train.ix[ite, :])
        xgbs.append(bst)
        '''
        
        # train
        lgb = lgbm.sklearn.LGBMClassifier(n_estimators=1500, seed=0, **params)
        lgb.fit(X_train.ix[itr, :], y_train.ix[itr])
        ypred = lgb.predict_proba(X_train.ix[ite, :])[:, 1]

        '''
        model.fit(X_train.ix[itr, :].values, y_train_cat[itr],
            epochs=60,
            batch_size=1000)
        ypred = model.predict(X_train.ix[ite, :].values)
        '''
    #ypred = sum(ypred) / len(ypred) 0.401408 0.392476
    pred_train[ite] = ypred

    
    sc.append(log_loss(y_train.ix[ite, :], pred_train[ite]))

    
print('XGB: {:.3f} +- {:.3f}'.format(np.mean(sc), np.std(sc)))

/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected.

XGB: 0.226 +- 0.002


In [113]:
def foo(x):
    a = 0.165 / 0.37
    b = (1 - 0.165) / (1 - 0.37) 
    return  a * x / (a * x + b * (1 - x))

In [127]:
pred_train = foo(pred_train)

In [128]:
pred_train = pd.DataFrame(pred_train)
pred_train.columns = ['y']
pred_train.to_csv("stacking/lgb_19.csv", index=None)

In [91]:
del X_train

NameError: name 'X_train' is not defined

In [104]:
import gc
gc.collect()

71

In [119]:
submit = pd.read_csv("data/sample_submission.csv")

In [65]:
#LGBM
params = {
    #'task': 'train',
    #'boosting_type': 'gbdt',
    #'objective': 'regression',
    #'metric': {'l2', 'auc'},
    #'num_leaves': 31,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    #'subsample': 0.8,
    #'colsample_bytree': 0.7,
    'objective': 'binary',
    'nthread': -1,
    'learning_rate': 0.1,
    'min_child_weight': 100,
    #'max_depth': 6
    #'num_class': 3
    
}

In [66]:
lgb = lgbm.sklearn.LGBMClassifier(n_estimators=1500, seed=0, **params)
lgb.fit(X_train, y_train)


/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=255, max_depth=-1,
        max_drop=50, min_child_samples=10, min_child_weight=100,
        min_split_gain=0, n_estimators=1500, nthread=-1, num_leaves=31,
        objective='binary', reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
        seed=0, sigmoid=1.0, silent=True, skip_drop=0.5, subsample=1,
        subsample_for_bin=50000, subsample_freq=1, uniform_drop=False,
        xgboost_dart_mode=False)

In [100]:
X_test = X_test.ix[1000001:, :]

In [101]:
test_pred2 = lgb.predict_proba(X_test)[:, 1]

In [98]:
test_pred1[:4]

array([ 0.00238842,  0.16396599,  0.48445084,  0.00186383])

In [102]:
test_pred1[-4:]

array([  2.82320214e-04,   7.93127424e-02,   5.11417772e-01,
         1.19566213e-04])

In [103]:
test_pred2[:4]

array([  2.04161744e-01,   5.60700423e-05,   1.55585459e-04,
         9.90305652e-02])

In [112]:
test_pred2[-4:]

array([  3.54776044e-04,   5.87363544e-05,   4.17804929e-01,
         3.63192961e-01])

In [105]:
len(test_pred1)

1000001

In [106]:
len(test_pred2)

1345795

In [107]:
test_pred = np.concatenate([test_pred1, test_pred2], axis=0)

In [108]:
len(test_pred)

2345796

In [114]:
test_pred

array([  2.38842223e-03,   1.63965988e-01,   4.84450842e-01, ...,
         5.87363544e-05,   4.17804929e-01,   3.63192961e-01])

In [115]:
del X_test

In [116]:
import gc
gc.collect()

125

In [38]:
param = {}
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'logloss'
param['max_depth'] = 7
# param['booster'] = 'dart'
#param['booster'] = 'gblinear'
param['eta'] = 0.1
#param['subsample'] = 1.0
# param["scale_pos_weight"] = 0.5
#param['colsample_bytree'] = 0.7
param['nthread'] = 8
#param['num_class'] =3
#param['alpha'] = 0.00
#param['lambda_bias'] = 0.1
# param['lambda'] = 0
#param['min_child_weight'] = 100

# if param['booster'] == 'gblinear': param['eta'] *= 1e-4
numround = 2000

In [39]:
Xdatatrain = xgb.DMatrix(data=X_train.ix[:, :].values,
                                     label=y_train.ix[:].values)

In [40]:
Xdataval = xgb.DMatrix(data=X_train.ix[:10, :].values,
                                     label=y_train.ix[:10].values)

In [62]:
del Xdatatest2

In [57]:
Xdatatest2 = xgb.DMatrix(data=X_test.ix[1000001:, :].values)

In [42]:

plst = list(param.items())
watchlist = [(Xdatatrain, 'train'), (Xdataval, 'eval')]

In [45]:
bst = xgb.train(plst, Xdatatrain, numround, evals=watchlist, verbose_eval=15)


[0]	train-logloss:0.633506	eval-logloss:0.646774
[15]	train-logloss:0.323969	eval-logloss:0.388133
[30]	train-logloss:0.271702	eval-logloss:0.343325
[45]	train-logloss:0.257076	eval-logloss:0.341127
[60]	train-logloss:0.248104	eval-logloss:0.316478
[75]	train-logloss:0.241793	eval-logloss:0.306781
[90]	train-logloss:0.236545	eval-logloss:0.294224
[105]	train-logloss:0.232073	eval-logloss:0.288613
[120]	train-logloss:0.228825	eval-logloss:0.282442
[135]	train-logloss:0.225883	eval-logloss:0.283732
[150]	train-logloss:0.222672	eval-logloss:0.279839
[165]	train-logloss:0.220304	eval-logloss:0.271811
[180]	train-logloss:0.218097	eval-logloss:0.271271
[195]	train-logloss:0.215673	eval-logloss:0.269905
[210]	train-logloss:0.213894	eval-logloss:0.270819
[225]	train-logloss:0.211979	eval-logloss:0.273126
[240]	train-logloss:0.210103	eval-logloss:0.267782
[255]	train-logloss:0.208528	eval-logloss:0.267423
[270]	train-logloss:0.207038	eval-logloss:0.269102
[285]	train-logloss:0.205572	eval-loglo

In [61]:
test_pred2 = bst.predict(Xdatatest2)


In [54]:
test_pred1[:4]

array([  4.93649067e-03,   1.69867009e-01,   5.44634938e-01,
         2.05250995e-04], dtype=float32)

In [50]:
len(test_pred1)

1000001

In [53]:
test_pred1[:4]

array([  2.29527755e-03,   5.45756638e-01,   6.37812853e-01,
         4.23383579e-04], dtype=float32)

In [64]:
test_pred = np.concatenate([test_pred1, test_pred2], axis=0)

In [65]:
len(test_pred)

2345796

In [117]:
test_pred = foo(test_pred)

In [120]:
submit['is_duplicate'] = test_pred

In [121]:
submit.to_csv("stacking/lgb_19_test.csv", index=None)

In [122]:
submit

,test_id,is_duplicate
0,0,0.000805
1,1,0.061903
2,2,0.240218
3,3,0.000628
4,4,0.040533
5,5,0.001306
6,6,0.997307
7,7,0.196047
8,8,0.341522
9,9,0.001433


In [161]:
submit

,test_id,is_duplicate
0,0,0.002633
1,1,0.323567
2,2,0.218058
3,3,0.000561
4,4,0.095342
5,5,0.007831
6,6,0.956379
7,7,0.098221
8,8,0.363882
9,9,0.002275
